In [1]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [2]:
filename = "ABox.ttl"

In [3]:
text1 = '''CQ_1.1
What are the cultural resources whose creation involved some author?
'''

query1 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?resource ?agent
WHERE {
    ?creation_event a frbroo:F28_Expression_Creation ;
            frbroo:R17_created ?resource ;
            crm:P9_consists_of ?activity .
    ?activity crm:P2_has_type aat:300054698 ;
            crm:P14_carried_out_by ?agent .
}
'''

In [4]:
text2 = '''CQ_1.2
What are the agents and the types of  activities they carried out to contribute to the creation of `32-exp`?
'''

query2 = '''
PREFIX ex: <https://w3id.org/dharc/ontology/chad-ap/data/example/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?agent ?type
WHERE {
    ?creation_event a frbroo:F28_Expression_Creation ;
            frbroo:R17_created ex:32-exp ;
            crm:P9_consists_of ?activity .
    ?activity crm:P2_has_type ?type ;
            crm:P14_carried_out_by ?agent .
}
'''

In [5]:
text3 = '''CQ_1.3
What are the agents involved in the creation of the cultural resources that have been created through the drawing technique? What are  the types of the activities they carried out?
'''

query3 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>  

SELECT ?resource ?agent ?type
WHERE {
    ?creation_event a frbroo:F28_Expression_Creation ;
            crm:P32_used_general_technique aat:300033973 ;
            frbroo:R17_created ?resource ;
            crm:P9_consists_of ?activity .
    ?activity crm:P2_has_type ?type ;
            crm:P14_carried_out_by ?agent .
}
'''

In [6]:
text5 = '''CQ_2.1
What are the identifiers of the manuscript? What are their types?
'''

query5 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?resource ?identifier_text ?type
WHERE {
    ?resource a frbroo:F4_Manifestation_Singleton ;
                crm:P2_has_type aat:300028569 ;
                crm:P48_has_preferred_identifier ?identifier .
    ?identifier crm:P2_has_type ?type ;
                crm:P190_has_symbolic_content ?identifier_text.
}
'''

In [7]:
text6 = '''CQ_2.2
What is the shelf mark of the cultural resource with the project ID "32"?
'''

query6 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?identifier_text
WHERE {
    ?resource a frbroo:F4_Manifestation_Singleton ;
    crm:P48_has_preferred_identifier ?identifier1 ,
                      ?identifier2 .
    ?identifier1 crm:P2_has_type aat:300312355 ;
                 crm:P190_has_symbolic_content "32" .
    ?identifier2 crm:P2_has_type aat:300404704 ;
                 crm:P190_has_symbolic_content ?identifier_text.
}
'''

In [8]:
text7 = '''CQ_2.3
What are the descriptive labels of the cultural resources that either have shelf marks or are prints?
'''

query7 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?resource ?label
WHERE {
    ?resource a frbroo:F4_Manifestation_Singleton ;
                crm:P48_has_preferred_identifier ?identifier ;
                crm:P2_has_type ?type ;
                crm:P3_has_note ?label .
    ?identifier crm:P2_has_type ?id_type .
    FILTER(?type = aat:300041273 || ?id_type = aat:300404704)
}
'''

In [9]:
text8 = '''CQ_3.1
What are the cultural resources whose collections have been curated by agents located in Bologna?
'''

query8 = '''
PREFIX ex: <https://w3id.org/dharc/ontology/chad-ap/data/example/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 

SELECT ?resource ?agent
WHERE {
    ?curation a crm:E87_Curation_Activity ;
            crm:P14_carried_out_by ?agent ;
            crm:P12_occurred_in_the_presence_of ?resource .
    ?agent crm:P74_has_current_or_former_residence ex:bologna .
}
'''

In [10]:
text10 = '''CQ_4.1
What are the cultural resources and the time spans of their conception events?
'''

query10 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/> 

SELECT ?resource ?time_interval
WHERE {
    ?conception_event a frbroo:F27_Work_Conception ;
                        frbroo:R16_initiated ?resource ;
                        crm:P4_has_time-span ?time_interval .
}
'''

In [11]:
text12 = '''CQ_5.1
What are the titles of the work `32-work`? What are their types?
'''

query12 = '''
PREFIX ex: <https://w3id.org/dharc/ontology/chad-ap/data/example/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>  

SELECT ?title ?type ?content
WHERE {
    ?creation a frbroo:F28_Expression_Creation ;
              frbroo:R19_created_a_realisation_of ex:32-work .
    ex:32-work crm:P102_has_title ?title .
    ?title crm:P2_has_type ?type ;
                crm:P190_has_symbolic_content ?content .
}
'''

In [12]:
text15 = '''CQ_5.4
What are the parent works of the works that have either 'ermafrodita' as their subject?
'''

query15 = '''
PREFIX ex: <https://w3id.org/dharc/ontology/chad-ap/data/example/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>  

SELECT ?parent ?work ?subject
WHERE {
  ?creation a frbroo:F28_Expression_Creation ;
              frbroo:R19_created_a_realisation_of ?work ;
              frbroo:R17_created ?expression .
    ?parent a frbroo:F15_Complex_Work ;
            frbroo:R10_has_member ?work .
    ?expression a frbroo:F2_Expression ;
                crm:P129_is_about ex:sub-ermafrodita .
}
'''

In [13]:
text18 = '''CQ_5.7
Which license statements are assigned to the manifestations?
'''

query18 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 

SELECT ?manifestation ?license_link
WHERE {
  ?license crm:P67_refers_to ?manifestation ;
    crm:P2_has_type aat:300435434 ;
    crm:P70i_is_documented_in ?license_link .
}
'''

In [14]:
text19 = '''CQ_1.1
What is the cultural object digitized and the digital object produced by the digitization process? What is the latter's license?
'''

query19 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT ?input ?output ?license_link
WHERE {
    ?activity a crmdig:D2_Digitization_Process ;
        crmdig:L1_digitized ?input ;
        crmdig:L11_had_output ?output .
    ?license crm:P2_has_type aat:300435434 ;
        crm:P67_refers_to ?output ;
        crm:P70i_is_documented_in ?license_link .
}
'''

In [15]:
text20 = '''CQ_1.2
What are the starting and ending date times in which the digitization process and the following software activity occurred?
'''

query20 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/> 

SELECT ?activity ?start ?end
WHERE {
    ?activity a crmdig:D2_Digitization_Process;
        crm:P4_has_time-span ?timespan .
    ?timespan crm:P82a_begin_of_the_begin ?start ;
        crm:P82b_end_of_the_end ?end .
}
'''

In [16]:
text21 = '''CQ_2.1
What are the digitization process, its output, the processing activity and its output?
'''

query21 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>

SELECT ?digitization ?input ?activity ?output
WHERE {
    ?activity a crmdig:D10_Software_Execution ;
    crm:P2_has_type aat:300054636 ;
    crmdig:L10_had_input ?input ;
    crmdig:L11_had_output ?output .
    ?digitization crmdig:L11_had_output ?input .
}
'''

In [17]:
text22 = '''CQ_2.2
What are the people and institutions who either carried out or were participants in the processing activity?
'''

query22 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>

SELECT ?person ?institution
WHERE {
    ?activity a crmdig:D10_Software_Execution ;
    crm:P2_has_type aat:300054636 ;
    crm:P14_carried_out_by ?person ;
    crm:P11_had_participant ?institution .
}
'''

In [18]:
text23 = '''CQ_3.1
What are the techniques used in digitization processes?
'''

query23 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>

SELECT ?technique ?activity
WHERE {
    ?activity a crmdig:D2_Digitization_Process ;
        crm:P32_used_general_technique ?technique .
}
'''

In [19]:
text24 = '''CQ_3.2
What are the tools and their types used in processing activities?
'''

query24 = '''
PREFIX aat: <http://vocab.getty.edu/page/aat/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>

SELECT ?tool ?type
WHERE {
    ?activity a crmdig:D10_Software_Execution ;
        crm:P2_has_type aat:300054636 ;
        crmdig:L23_used_software_or_firmware ?tool .
    ?tool a crmdig:D14_Software ;
        crm:P2_has_type ?type .
}
'''

In [20]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3),
           (text5, query5),
           (text6, query6),
           (text7, query7),
           (text8, query8),
           (text10, query10),
           (text12, query12),
           (text15, query15),
           (text18, query18),
           (text19, query19),
           (text20, query20),
           (text21, query21),
           (text22, query22),
           (text23, query23),
           (text24, query24)]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))
    print("\n")

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x0000018BBD99BA60>
Traceback (most recent call last):
  File "c:\Users\Utente\Documents\Projects\cidoc-profile\venv\Lib\site-packages\isodate\isodatetime.py", line 51, in parse_datetime
    datestring, timestring = datetimestring.split('T')
    ^^^^^^^^^^^^^^^^^^^^^^
ValueError: not enough values to unpack (expected 2, got 1)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Utente\Documents\Projects\cidoc-profile\venv\Lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\Utente\Documents\Projects\cidoc-profile\venv\Lib\site-packages\isodate\isodatetime.py", line 53, in parse_datetime
    raise ISO8601Error("ISO 8601 time designator 'T' missing. Unable to"
isod

CQ_1.1
What are the cultural resources whose creation involved some author?

+-------------------------------------------------------------+------------------------------------------------------------------------+
| resource                                                    | agent                                                                  |
|-------------------------------------------------------------+------------------------------------------------------------------------|
| https://w3id.org/dharc/ontology/chad-ap/data/example/32-exp | https://w3id.org/dharc/ontology/chad-ap/data/example/ulisse-aldrovandi |
+-------------------------------------------------------------+------------------------------------------------------------------------+


CQ_1.2
What are the agents and the types of  activities they carried out to contribute to the creation of `32-exp`?

+------------------------------------------------------------------------+-------------------------------------------+
